# Friendship paradox

The friendship paradox is an interesting phenomenon that highlights a disparity between an individual's perception of their own popularity and the average popularity of their friends. In simple terms, the friendship paradox states that, on average, your friends will have more friends than you do. This may seem counterintuitive at first, as we tend to believe that we are just as popular as our friends. However, when we consider from network perspective, it becomes clear why this paradox exists.

To understand the friendship paradox, let's analyze the Facebook network.

In [ ]:
import pandas as pd

edge_table = pd.read_csv("https://raw.githubusercontent.com/skojaku/adv-net-sci-course/main/data/fb_friends_draw.csv")
edge_table

Let's see the degree distribution of the networks.  We first construct an adjacency matrix. We will use `igraph`'s convenient API to construct the adjacency matrix from the edge table.

In [ ]:
import numpy as np
from scipy import sparse
import igraph

g = igraph.Graph.DataFrame(
    edge_table[["src", "trg"]],
    directed=False,
)
A = g.get_adjacency_sparse()

Then, compute the degree of each node by taking row sum of the matrix. While the igraph has an API to compute the degree sequence, we will use the adjacency matrix as exercise.

In [ ]:
# Assignment:
# Create a numpy array named `deg`, where deg[i] indicates the degree of node i

Then, plot the distribution of the degree in two ways. First, create the histogram (probability mass function; PMF) of the node degree. Don't forget to label the axes.

In [ ]:
# Assignment. Create the histogram of the degree distributon.
import seaborn as sns
import matplotlib.pyplot as plt


You will see a skewed distribution, where most nodes have a small degree, while few nodes have exceptionally large degree (hubs). While the hubs are minority in the network, they hold most edges in the network and thus often play critical roles in the network structure.

Another way to visualize the degree distribution is the cumulative distribution. It provides information about the probability of a random variable taking on a value less than or equal to a given value. Mathematically, the cumulative probability distribution of a random variable X is defined as:

$$F(x) = P(X \leq x)$$

where $F(x)$ represents the cumulative probability of $X$ being less than or equal to $x$.

The cumulative probability distribution can be represented graphically using a cumulative distribution curve. This curve starts at 0 for the smallest possible value of X and ends at 1 for the largest possible value of X. The curve is non-decreasing, as the probability of $X$ being less than or equal to a value increases as that value increases.

Additionally, by using the logarithmic scale, we can plot extremely small and large values more clearly. Let's see how the CDF looks like in the log-log plot.

In [ ]:
sns.set_style("white")
sns.set(font_scale=1.2)
sns.set_style("ticks")
fig, ax = plt.subplots(figsize=(4, 4))

ax = sns.ecdfplot(data=deg, stat="proportion")

ax.set_xlabel("Degree")
ax.set_ylabel("Cumulative probability")

# Specify the scale to be logarithmic
ax.set_xscale("log")
ax.set_yscale("log")

sns.despine()

Another variant of the cumulative distribution, and the one that is more useful for visualizing the degree distribution, is the "complementary" cumulative distribution function (CCDF).

In [ ]:
sns.set_style("white")
sns.set(font_scale=1.2)
sns.set_style("ticks")
fig, ax = plt.subplots(figsize=(4, 4))

ax = sns.ecdfplot(data=deg, stat="proportion", complementary=True)

ax.set_xlabel("Degree")
ax.set_ylabel("Cumulative probability")

ax.set_xscale("log")
ax.set_yscale("log")

sns.despine()

So, when to use PDF, CDF, or CCDF? PDF is the most direct representation of the distribution and probably the most easily understood by public audience. Unless the distribution is highly skewed, PDF suffices in many cases. If your distribution is highly skewed or you want to show the extreme values, CDF and CCDF (+logarithmic scale) are useful. In particular, CCDF is highly useful to see the right tail of a distribution.

Armed with the visualization tools, let's approach the friendship paradox. The friendship paradox dictates that the friends have more friends more than average. Let's compute the average degree.

In [ ]:
average_deg = np.mean(deg)
average_deg

Then, compute the average degree (the number of friends) of a friend:

In [ ]:
# Assignment: compute the average number of friends a friend of a node has:
# Hint:
# 1. Go through each node and then go through each neighbor of the node. This is the same as going through individual edges.
# 2. Append the degree of the neighbor to a list
# 3. Once you go through every neighbor of every node, compute the average of the degree in the list.

degree_of_friends = []
#
# Write your code ----
#


# --------------------
average_degree_of_neighbors = np.mean(degree_of_friends)
average_degree_of_neighbors

Do you witness the friendship paradox? Plot the distribution of the friends' degrees by using CCDF.

In [ ]:
sns.set_style("white")
sns.set(font_scale=1.2)
sns.set_style("ticks")
fig, ax = plt.subplots(figsize=(4, 4))

ax = sns.ecdfplot(data=deg, stat="proportion", complementary=True, label="Degree")
ax = sns.ecdfplot(
    data=degree_of_friends,
    stat="proportion",
    complementary=True,
    label="Friend's degree",
)
ax.set_xlabel("Degree")
ax.set_ylabel("Cumulative probability")

ax.set_xscale("log")
ax.set_yscale("log")

ax.legend(frameon=False)

sns.despine()

The plot shows that the friend's degree distribution is more right skewed, suggesting the friendship paradox, i.e., friends tend to have more friends than on average.

So why does the friendship paradox arise? When we pick a friend of a node, we essentially traverse an edge from the node, and most edges are connected to popular individuals (hubs), since they hold most edges in the network. In other words, popular individuals tend to appear the friend list of many people, and these popular individuals are counted multiple times, when computing the degree of friends, resulting in the friendship paradox.

# Counting paths and triangles

Our expansive social network comprises billions of individuals, with each person being linked to every other individual on Earth through a series of friendship connections, typically no more than six steps apart. This characteristic, where all nodes within a network are connected by relatively short paths, is referred to as the small-world property. Remarkably, this phenomenon is observed in various types of networks spanning different fields.

While we have a small social network of hundreds of people, it is still useful to exercise how to compute the number of paths and lengths. We will compute *network diameter*, the maximum length of the shortest path in the network.

With the adjacency matrix, you can compute the number of paths between any two nodes by using a simple matrix operation. Let's remind that an element $A[i,j]$ of adjacency matrix $A$ represents the presence and absence of an edge between $i$ and $j$. An edge is interpreted as a path of length 1, right? Now, let's think about the product of two matrices, $B:=A \times A$. Can you guess what $B[i,j]$ represents?

In [ ]:
B = A @ A
B.toarray()

The element $B[i,j]$ represents the number of paths of length $d=2$ that connect nodes $i$ and $j$. The diagonal element $B[i,i]$ corresponds to the node degree, since it is counting a walk of length 2 between a node and its neighbor.

But why the product of the adjacency matrices gives you the path length of 2? Let's describe the element mathematically.
$$
B_{ij} = \sum_{k} A_{ik} A_{kj}
$$
It is essentially a sum of products, $A_{ik}A_{kj}$, and each product $A_{ik}A_{kj}$ takes one if and only if $A_{ik}=A_{kj}=1$, i.e., node $i$ is connected to node $k$ and $k$ is connected to node $j$. In other words, $A_{ik}A_{kj}$ represents a sequence of nodes $(i,k,j)$ and is on (=1) when they form a path. And $B_{ij}$ is counting up all the connected node sequences of length three.

We can extend this idea to the case of three path lengths, four path lengths, and more. Simply multiplying the adjacency matrix $d$ gives you a count of paths of length $d$.

Armed with this property, can you count the number of triangles in the network?

In [ ]:
# Assignment: Compute the number of triangles in the network
# Hint:
# - Triangle is a path of length three that starts and ends at the same node.
# - Don't double count the same triangle!

Finally, let's compute the network diameter. The network diameter is the maximum length of the shortest paths connecting two nodes in the network.

To determine the network diameter, we need to find the longest shortest path in the network. One approach is to start with a path length of $d=2$ and increment $d$ by one until all nodes are connected by paths of length $d$ or less. Here is a step-by-step strategy:

1. Begin with a path length of $d=2$.
2. Check if there is a path of length $d$ or less between every pair of nodes.
3. If there are any two nodes that are not connected by a path of length $d$ or less, it means we have not reached the diameter yet. In this case, increase the path length $d$ and repeat step 2.

In [ ]:
# Assignment: Compute the network diameter
d